# Make movie for Result/Nanoparticle slide

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
# Peak detection
from scipy.spatial import cKDTree as KDTree
import sys
import os
import tempfile
import hyperspy.api as hs
import skimage.io
from stm.feature.peaks import find_local_peaks, refine_peaks
from skimage.morphology import disk
from scipy.spatial import cKDTree as KDTree
from stm.preprocess import normalize


In [ ]:
imgroot = 'experiment/Au-CeO2_HVAC14'
atomsroot = imgroot + '_atoms_negdefocus'
predroot = imgroot + "_prediction_negdefocus"
moviename = "movie_thomas_negdefocus.m4v"
sampling = 0.105
threshold = None

In [ ]:
def namegenerator(imgroot, dataroot, predroot):
    for root, dirs, files in os.walk(imgroot):
        dirs.sort()
        files.sort()
        for f in files:
            fullname = os.path.join(root, f)
            f_noext, ext = os.path.splitext(fullname)
            if ext not in ['.dm4', '.png']:
                continue
            assert f_noext.startswith(imgroot)
            dataname = dataroot + f_noext[len(imgroot):] + '_atoms.npy'
            predname = predroot + f_noext[len(imgroot):] + '_prediction.npz'
            yield fullname, dataname, predname

In [ ]:
names = namegenerator(imgroot, atomsroot, predroot)

In [ ]:
def get_image(f):
    if f.endswith('.dm4'):
        a = hs.load(f)
        image = a.data
    else:
        a = skimage.io.imread(f)
        image = skimage.color.rgb2gray(a)
    return image

def get_points(f):
    atoms = np.load(f)
    return atoms

def get_prediction(f):
    pred = np.load(f)['prediction']
    return pred

In [ ]:
def crop(image1, image2, atoms, xmin=0, xmax=None, ymin=0, ymax=None):
    shape = image1.shape
    if xmax is None:
        xmax = shape[0]
    if ymax is None:
        ymax = shape[1]
    image1 = image1[xmin:xmax,ymin:ymax]
    if image2 is not None:
        image2 = image2[xmin:xmax,ymin:ymax]

    d = 7
    atoms = atoms[np.where(atoms[:,0] > xmin + d)]
    atoms = atoms[np.where(atoms[:,1] > ymin + d)]
    atoms = atoms[np.where(atoms[:,0] < xmax - d)]
    atoms = atoms[np.where(atoms[:,1] < ymax - d)]
    atoms = atoms - np.array((xmin, ymin))
    return image1, image2, atoms

In [ ]:
def find_peaks_again(prediction):
    distance = int(2.5 / sampling)
    peaks = find_local_peaks(prediction, min_distance=distance, 
                             threshold=threshold, exclude_border=10,
                             exclude_adjacent=True)
    peaks = refine_peaks(normalize(prediction), peaks, 
                         disk(2), model='polynomial')
    return peaks

In [ ]:
def makefigure(image, atoms, prediction, plotfile=None):
    image, prediction, atoms = crop(image, prediction, atoms, xmin=500, xmax=1150, ymin=150, ymax=800)

    fig, (ax_raw, ax_pred, ax_atoms) = plt.subplots(1,3,figsize=(20,11))
    #fig, (ax_raw, ax_pred, ax_atoms) = plt.subplots(3,1,figsize=(12,40))

    ax_raw.imshow(image,cmap='gray')
    ax_pred.imshow(prediction,cmap='gray')
    ax_atoms.imshow(image, cmap='gray')
    #mycolor = '#80FFFF'
    mycolor = '#FFFF00'
    ax_atoms.scatter(atoms[:,1], atoms[:,0], c=mycolor, marker='o', linewidth=2.0)
    ax_raw.axis('off')
    ax_pred.axis('off')
    ax_atoms.axis('off')
    plt.tight_layout()
    if plotfile:
        fig.savefig(plotfile, bbox_inches='tight')
        plt.close(fig)

## Run this section only for draft

In [ ]:
fi, fd, fp = next(names)
print(fi)
image = get_image(fi)
atoms = get_points(fd)
prediction = get_prediction(fp)

In [ ]:
atoms2 = find_peaks_again(prediction)

In [ ]:
makefigure(image, atoms, prediction)

## Run this section to produce movie

In [ ]:
#with tempfile.TemporaryDirectory() as tmpdir:
tmpdirectory = tempfile.TemporaryDirectory()
tmpdir = tmpdirectory.name
filenames = os.path.join(tmpdir, 'files.txt')
with open(filenames, "wt") as f:
    print("Temporary folder:", tmpdir)
    for i, (fi, fd, fp) in enumerate(names):
        image = get_image(fi)
        atoms = get_points(fd)
        prediction = get_prediction(fp)
        print(fi)
        if threshold:
            atoms = find_peaks_again(prediction)
        pngfile = 'a{:04d}.png'.format(i)
        plotfile = os.path.join(tmpdir, 'a{:04d}.jpg'.format(i))
        tfile = os.path.join(tmpdir, 'tmp_'+pngfile)
        makefigure(image, atoms, prediction, tfile)
        os.system("convert -trim -quality 100 {} {}".format(tfile, plotfile))
        #os.remove(tfile)
        f.write(plotfile + '\n')
        #if i == 25: break

The images are converted to a movie using the `convert` utility from ImageMagick.  Unfortunately, it produces a video that PowerPoint cannot play, so it has to be converted again by a video conversion utility (I use Wondershare for Mac).

In [ ]:
cmd = 'convert -delay 5 -quality 95 @{} {}'.format(filenames, moviename)
print(cmd)
os.system(cmd)

In [ ]:
# del tmpdirectory